In [0]:
pip install prophet

In [0]:
# imports
import pandas as pd
import numpy as np
import pytz
from datetime import datetime, timedelta, time
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pyspark.sql.functions import to_timestamp
from prophet.plot import plot_forecast_component
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, StructType, DoubleType, LongType
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics,BinaryClassificationMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, to_timestamp, lit, udf
from pyspark.ml import Pipeline

In [0]:
team_BASE_DIR = f"dbfs:/student-groups/Group_4_1"
spark.sparkContext.setCheckpointDir(f"{team_BASE_DIR}/modeling_checkpoints")
display(dbutils.fs.ls(f"{team_BASE_DIR}"))

In [0]:
# TODO: replace with Shruti's imputed weather df once available

period = "1y" # on of the following values ("", "3m", "6m", "1y")
df = spark.read.parquet(f"{team_BASE_DIR}/interim/join_checkpoints/joined_flights_weather_{period}_v1.parquet")

In [0]:
len(df.dtypes)

In [0]:
df.count()

In [0]:
# define train/test split and cross-validation folds
min_test_dt = "2019-10-01" # reserve last 3 months of year for held out test set
k = 3

# define var names
dep_utc_varname = "sched_depart_utc"

In [0]:
# convert time variable to datetime
df = df.withColumn(dep_utc_varname, to_timestamp(col(dep_utc_varname)))

# add hour variable (needed for seasonality)
df = df.withColumn("dep_hour_utc", f.hour(col(dep_utc_varname)))

# define outcome variable
df = df.withColumn("outcome", (when((col("DEP_DELAY") >= 15) | (col("CANCELLED") == 1), 1).otherwise(0)).cast("double"))


In [0]:
# define train and test data
df_train = df.filter(f.col(dep_utc_varname) < min_test_dt)
df_train.cache()
# df_train.checkpoint()
df_test = df.filter(f.col(dep_utc_varname) >= min_test_dt)
df_train.cache()
# df_train.checkpoint()

In [0]:
# CODE IN THIS CELL DERIVED FROM DEMO 11 NOTEBOOK

def get_cv_time_limits(df, k=3, blocking=False, dep_utc_varname="dep_datetime", verbose=True):
    '''
    Get time bins for time-series cross validation
    '''
    n = df.count()
    df = df.withColumn("row_id", f.row_number()
            .over(Window.partitionBy().orderBy(dep_utc_varname)))
    chunk_size = np.floor(n/(k+1))

    idx = np.arange(0,)
    idx = np.arange(0,n,chunk_size)
    idx[-1] = n-1
    idx = [int(i)+1 for i in idx]
    
    if verbose:
        print('')
        print(f'Number of validation datapoints for each fold is {chunk_size:,}')
        print("************************************************************")

    bin_edges = df.filter(f.col("row_id").isin(idx)).select("row_id",dep_utc_varname).toPandas()

    out = []
    for i in range(k):
        # define minimum training time based on cross-validation style
        if not blocking:
            t_min_train = bin_edges[dep_utc_varname][0]
        else:
            t_min_train = bin_edges[dep_utc_varname][i]
        # define maximum training time
        t_max_train = bin_edges[dep_utc_varname][i+1]
        # define minimum test time
        t_min_test = bin_edges[dep_utc_varname][i+1]
        # define maximum test_time
        t_max_test = bin_edges[dep_utc_varname][i+2]

        out.append({"train_min":t_min_train, "train_max":t_max_train,
                    "test_min":t_min_test, "test_max":t_max_test})
    out = pd.DataFrame(out)
        
    if verbose:
        for i in range(k):
            print(f'    TRAIN set for fold {i} goes from {out["train_min"][i]} to {out["train_max"][i]}')
            print(f'    TEST set for fold {i} goes from {out["test_min"][i]} to {out["test_max"][i]}')
        
    return out

In [0]:
# get cross-validation split times
cv_cutoffs = get_cv_time_limits(df_train.select(dep_utc_varname), k=3, blocking=True, 
    dep_utc_varname=dep_utc_varname, verbose=True)
cv_cutoffs

## Train seasonality models for each fold

In [0]:
# informed by: https://www.databricks.com/blog/2021/04/06/fine-grained-time-series-forecasting-at-scale-with-facebook-prophet-and-apache-spark-updated-for-spark-3.html

def forecast_delay(history_pd: pd.DataFrame) -> pd.DataFrame: 
    
    model = Prophet(
        interval_width=0.9,
        growth='linear',
        weekly_seasonality=True,
        daily_seasonality=True,
        yearly_seasonality=True,
        # holidays=us_holidays,
        # seasonality_mode='multiplicative'
    )

    # TODO: add dummy row to history_pd so that timestamps end up as round hours
    
    # fit the model
    model.fit(history_pd)
    
    # configure predictions
    future_pd = model.make_future_dataframe(
        periods=24*7, 
        freq='h',
        include_history=False
    )
    
    # make predictions
    results_pd = model.predict(future_pd)

    # ref date and dow
    ref_date = history_pd.ds.iloc[0].date()
    ref_dow = history_pd.DAY_OF_WEEK[0]

    def get_dow(x,ref_date,dow):
        d_days = (x.date() - ref_date).days + dow
        d_days = d_days%7
        if d_days == 0:
            d_days = 7
        return d_days

    # dateshift
    results_pd['dow'] = results_pd.ds.apply(lambda x: get_dow(x,ref_date,ref_dow))

    # hour
    results_pd['hour'] = results_pd.ds.apply(lambda x: x.hour)

    # apply origin
    results_pd['ORIGIN'] = history_pd.ORIGIN.iloc[0]

    # # get seasonality
    # results_pd['seasonality'] = results_pd['weekly'] + results_pd['daily']
        
    # return predictions
    return results_pd[['dow','hour','weekly','daily','ORIGIN']]

schema = StructType([StructField('dow', LongType(), True),
                     StructField('hour', LongType(), True),
                     StructField('weekly', DoubleType(), True),
                     StructField('daily', DoubleType(), True),
                     StructField('ORIGIN', StringType(), True)])

In [0]:
def get_seasonality( df, t_min, t_max, 
    dep_utc_varname=dep_utc_varname, delay_varname="DEP_DELAY", 
    forecast_fn=forecast_delay, schema=schema ):

    return (
        df.filter((df[dep_utc_varname] >= t_min) & \
            (df[dep_utc_varname] < t_max))
        .withColumnRenamed(delay_varname, "y")
        .withColumnRenamed(dep_utc_varname, "ds")
        .groupBy('ORIGIN')
            .applyInPandas(forecast_fn, schema=schema)
        )

In [0]:
# !!! SEASONALITY ALREADY RUN !!!

# # train seasonality model for each cross validation split
# for i in range(k):
#     # train seasonality model for this cross validation split
#     model = get_seasonality(df_train, cv_cutoffs["train_min"][i], cv_cutoffs["train_max"][i])
#     # write out
#     fn_out = f"seasonality_model_{period}_cv{i}of{k}.parquet"
#     model.write.parquet(f"{team_BASE_DIR}/interim/{fn_out}")

# # train seasonality model for full training data
# model = get_seasonality(df_train, datetime(1970,1,1), min_test_dt)
# # write out
# fn_out = f"seasonality_model_{period}_train.parquet"
# model.write.parquet(f"{team_BASE_DIR}/interim/{fn_out}")

In [0]:
display(dbutils.fs.ls(f"{team_BASE_DIR}/interim"))

## Modeling

TODO: Separate out feature engineering process and save out feat eng. cv split data

In [0]:
display(dbutils.fs.ls(f"{team_BASE_DIR}/interim/join_checkpoints"))

In [0]:
# df = spark.read.parquet(f"{team_BASE_DIR}/interim/join_checkpoints/joined_{period}_weather_cleaned_test.parquet")
df = spark.read.parquet(f"{team_BASE_DIR}/interim/join_checkpoints/joined_{period}_weather_cleaned_combo.parquet")


In [0]:
df.count()

In [0]:
# Get data types of each column in the DataFrame
column_data_types = df.dtypes

# Display the data types
for column, data_type in column_data_types:
    print(f"Column: {column}, Data Type: {data_type}")

In [0]:
weather_cols = [col for col in df.columns if "origin_Hourly" in col]
remove_me = ["origin_HourlyPresentWeatherType","origin_HourlySkyConditions","origin_HourlyWindDirection"]
weather_cols = [c for c in weather_cols if c not in remove_me]
weather_cols

In [0]:
seasonality_cols = ["daily","weekly"]

In [0]:
# convert time variable to datetime
df = df.withColumn(dep_utc_varname, to_timestamp(col(dep_utc_varname)))

# add hour variable (needed for seasonality)
df = df.withColumn("dep_hour_utc", f.hour(col(dep_utc_varname)))

# define outcome variable
df = df.withColumn("outcome", (when((col("DEP_DELAY") >= 15) | (col("CANCELLED") == 1), 1).otherwise(0)).cast("double"))


In [0]:
# select relevant vars
tmp = df.select(*weather_cols,"DAY_OF_WEEK","ORIGIN","dep_hour_utc","outcome",dep_utc_varname,"OP_UNIQUE_CARRIER","ORIGIN_ICAO","DEST_ICAO")

for column in weather_cols:
    tmp = tmp.withColumn(column, col(column).cast("double"))

# # REMOVE ME !!! JUST FOR TESTING
# tmp = tmp.filter(f.col("ORIGIN") == "BOS")

# simplistic cleaning: TODO: use Shruti's output instead
tmp = tmp.fillna({col:0 for col in weather_cols})

tmp.cache()

In [0]:
tmp.dtypes

In [0]:
# define train and test data
df_train = tmp.filter(f.col(dep_utc_varname) < min_test_dt)
df_train.cache()
# df_train.checkpoint()
df_test = tmp.filter(f.col(dep_utc_varname) >= min_test_dt)
df_train.cache()
# df_train.checkpoint()

In [0]:
display(df)

In [0]:
# List of categorical columns to encode
categorical_columns = ["OP_UNIQUE_CARRIER", "ORIGIN_ICAO", "DEST_ICAO"]

# List to hold the stages of the pipeline
stages = []

# Index and encode each categorical column
for column in categorical_columns:
    indexer = StringIndexer(inputCol=column, outputCol=column + "_index",handleInvalid="keep")
    encoder = OneHotEncoder(inputCol=column + "_index", outputCol=column + "_vec",handleInvalid="keep")
    stages += [indexer, encoder]

features = [col + "_vec" for col in categorical_columns]+[*weather_cols,*seasonality_cols]
assembler = VectorAssembler(inputCols=features, outputCol="features", handleInvalid='keep') # !!! TODO: fill all nulls
scaler = StandardScaler(inputCol="features", \
    outputCol="features_scaled",withMean=True)
lr = LogisticRegression(featuresCol='features_scaled', \
    labelCol='outcome',maxIter=50)
pipeline = Pipeline(stages=stages+[assembler,scaler,lr])

In [0]:
def get_seasonality_data(df, fold, k):
    if fold == 'full':
        fn_model = f"seasonality_model_{period}_train.parquet"
    else:
        fn_model = f"seasonality_model_{period}_cv{fold}of{k}.parquet"
    model = spark.read.parquet(f"{team_BASE_DIR}/interim/{fn_model}")

    joined_df = df.join(model, 
                     (df["ORIGIN"] == model["ORIGIN"]) & 
                     (df["DAY_OF_WEEK"] == model["dow"]) & 
                     (df["dep_hour_utc"] == model["hour"]),
                     how="left").drop(model["ORIGIN"])
    
    return joined_df

# display(get_seasonality_data(df_train.limit(10), 0, 3))

In [0]:
# CODE IN THIS CELL DERIVED FROM DEMO 11 NOTEBOOK

def upsample(train_df,verbose=False):
  '''Upsamples train_df to balance classes'''
  #balance classes in train
  delay_count = train_df.filter(f.col("outcome") == 1).count()
  non_delay_count = train_df.filter(f.col("outcome") == 0).count()

  total = delay_count + non_delay_count
  keep_percent = non_delay_count / delay_count

  train_delay = train_df.filter(f.col('outcome') == 0)
  train_non_delay = train_df.filter(f.col('outcome') == 1).sample(withReplacement=True, fraction=keep_percent,seed=42)
  train_upsampled = train_delay.union(train_non_delay)
  return train_upsampled


def downsample(train_df,verbose=False):
  '''Downsamples train_df to balance classes'''
  #balance classes in train
  delay_count = train_df.filter(f.col("outcome") == 1).count()
  non_delay_count = train_df.filter(f.col("outcome") == 0).count()

  total = delay_count + non_delay_count
  keep_percent = delay_count / non_delay_count
  
  train_delay = train_df.filter(f.col('outcome') == 1)
  train_non_delay = train_df.filter(f.col('outcome') == 0).sample(withReplacement=False,fraction=keep_percent,seed=42)
  train_downsampled = train_delay.union(train_non_delay)
  return train_downsampled

def cv_eval(preds):
  """
  Input: transformed df with prediction and label
  Output: desired score 
  """
  rdd_preds_m = preds.select(['prediction', 'outcome']).rdd
  rdd_preds_b = preds.select('outcome','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['outcome'])))
  metrics_m = MulticlassMetrics(rdd_preds_m)
  metrics_b = BinaryClassificationMetrics(rdd_preds_b)
  F2 = np.round(metrics_m.fMeasure(label=1.0, beta=2.0), 4)
  pr = metrics_b.areaUnderPR
  return F2, pr

def timeSeriesSplitCV(df, pipeline, cv_info, sampling=None, metric='f2', verbose=True, dep_utc_varname=dep_utc_varname):
  '''
  Perform timSeriesSplit k-fold cross validation 
  '''

  k = len(cv_info)
  
  # Track score
  scores=[]
  
  # Start k-fold
  for i in range(k):
    
    # Create train set
    train_df = df.filter((df[dep_utc_varname] >= cv_info["train_min"][i]) & \
      (df[dep_utc_varname] < cv_info["train_max"][i])).cache()
      
    # Create dev set
    dev_df = df.filter((df[dep_utc_varname] >= cv_info["test_min"][i]) & \
      (df[dep_utc_varname] < cv_info["test_max"][i])).cache() 

    # Apply sampling on train if selected
    if sampling=='down':
      train_df = downsample(train_df)
      train_df = train_df.cache()
    elif sampling=='up':
      train_df = upsample(train_df)
      train_df = train_df.cache()
    # elif sampling=='weights':
    #   train_df = add_class_weights(train_df).cache()
      
    #print info on train and dev set for this fold
    if verbose:
      print('    TRAIN set for fold {} goes from {} to {}, count is {:,} flights ({})'.format((i+1), 
                                                                                      train_df.agg({dep_utc_varname:'min'}).collect()[0][0],
                                                                                      train_df.agg({dep_utc_varname:'max'}).collect()[0][0],
                                                                                      train_df.count(),
                                                                                      sampling + '-sampled' if sampling else 'no sampling'))
      print('    DEV set for fold {} goes from {} to {}, count is {:,} flights'.format((i+1), 
                                                                                      dev_df.agg({dep_utc_varname:'min'}).collect()[0][0],
                                                                                      dev_df.agg({dep_utc_varname:'max'}).collect()[0][0],
                                                                                      dev_df.count()))
      
    # TODO: remove once feat engineering applied outside
    train_df = get_seasonality_data(train_df, i, k)
    train_df = train_df.fillna({col:0 for col in ['daily','weekly']})
    dev_df = get_seasonality_data(dev_df, i, k)
    dev_df = dev_df.fillna({col:0 for col in ['daily','weekly']})

    # print(train_df.dtypes)
    # print(dev_df.dtypes)
        
    # Fit params on the model
    model = pipeline.fit(train_df)
    dev_pred = model.transform(dev_df)
    if metric=='f2':
      score = cv_eval(dev_pred)[0]
    elif metric=='pr':
      score = cv_eval(dev_pred)[1]
    scores.append(score)
    print(f'    Number of training datapoints for fold number {i+1} is {train_df.count():,} with a {metric} score of {score:.2f}') 
    print('------------------------------------------------------------')
  
  # Take average of all scores
  avg_score = np.average(scores)    
  print(f'Average {metric} score across all folds is {avg_score:.2f}')
  print("************************************************************")

  # # Train on full df
  # print('Training on full train dataset, and validating on dev dataset with best parameters from CV:')
  # print(best_parameters)
    
  # if verbose:
  #   print('    TRAIN set for best parameter fitted model goes from {} to {}, count is {:,} flights ({})'.format(train_df.agg({dep_utc_varname:'min'}).collect()[0][0],
  #                                                                                                    train_df.agg({dep_utc_varname:'max'}).collect()[0][0],
  #                                                                                                    train_df.count(),
  #                                                                                                    sampling + '-sampled' if sampling else 'no sampling'))
  return avg_score

In [0]:
timeSeriesSplitCV(df_train, pipeline, cv_cutoffs, sampling='down', metric='f2', verbose=True, dep_utc_varname=dep_utc_varname)

In [0]:
# final training and evaluation

df_train = downsample(df_train).cache()
df_train = get_seasonality_data(df_train, 'full', k).cache()
df_test = get_seasonality_data(df_test, 'full', k).cache()
model = pipeline.fit(df_train)
dev_pred = model.transform(df_test)
# get f2 score
score = cv_eval(dev_pred)[0]
print(score)ap

## Sandbox: Figures for Presentation

In [0]:

fn_model = f"seasonality_model_{period}_train.parquet"
model = spark.read.parquet(f"{team_BASE_DIR}/interim/{fn_model}")

In [0]:
display(model)

In [0]:
airport = 'BOS'
seas = model.filter(col('ORIGIN') == airport).toPandas()

fig,ax = plt.subplots(2,1,figsize=(10,5))
seas['x'] = seas['dow'] + seas['hour']/24
seas.sort_values('x',inplace=True)
seas.plot(x='x',y='daily',ax=ax[0],legend=False)
seas.plot(x='x',y='weekly',ax=ax[1],legend=False)
ax[0].set_xticks([])
ax[0].set_xlabel('')
ax[0].set_title(f'{airport} Seasonality Components\nTrained on Jan-Sep 2019 Data')
ax[1].set_xlabel('Day of week')
ax[0].set_ylabel('Daily component (minutes)')
ax[1].set_ylabel('Weekly component (minutes)')
plt.show()

In [0]:
flights = spark.read.parquet(f"{team_BASE_DIR}/raw/flightdelays/parquet_airlines_data_{period}")
weather = spark.read.parquet(f"{team_BASE_DIR}/interim/weather_{period}_checkpoint")

In [0]:
len(flights.dtypes)

In [0]:
flights.count()

In [0]:
len(weather.dtypes)

In [0]:
weather.count()

In [0]:
# Get the count of each value in the outcome variable
outcome_counts = df.groupBy("outcome").count()

display(outcome_counts)

In [0]:
df = get_seasonality_data(df, 'full', 3).cache()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Select the relevant columns and convert to Pandas DataFrame
selected_columns = ["daily", "weekly", "outcome"]
df_pd = df.select(selected_columns).toPandas()

# Calculate the correlation matrix
correlation_matrix = df_pd.corr(method='spearman')

In [0]:
# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Spearman Correlation Heatmap\nSeasonality Features, 2019 (1 Year) Data")
plt.show()